In [ ]:
# https://openrouter.ai/settings/keys

In [4]:
!rm -rf /usr/local/lib/python3.11/dist-packages/~ensorflow*
!rm -rf /usr/local/lib/python3.11/dist-packages/tensorflow*
!rm -rf /usr/local/lib/python3.11/dist-packages/keras*
!pip install --upgrade --force-reinstall protobuf==3.20.*


  Using cached protobuf-3.20.3-py2.py3-none-any.whl.metadata (720 bytes)
Using cached protobuf-3.20.3-py2.py3-none-any.whl (162 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires tqdm, which is not installed.
kaggle 1.7.4.5 requires tqdm, which is not installed.
dataproc-spark-connect 0.8.3 requires tqdm>=4.67, which is not installed.
dopamine-rl 4.1.2 requires tensorflow>=2.2.0, which is not installed.
dopamine-rl 4.1.2 requires tensorflow-probability>=0.13.0, which is not installed.
dopamine-rl 4.1.2 requires tqdm>=4.64.1, which is not installed.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.2 r

In [3]:
!rm -rf /usr/local/lib/python3.11/dist-packages/~qdm*


In [4]:
!pip install tqdm


In [5]:
!pip install langchain_community

In [6]:
!pip install langchain-experimental

In [7]:
!pip install pdfplumber

In [8]:
!pip install faiss-cpu

In [9]:
!git clone https://github.com/Jaish19/GenAI---RAG-using-LangChain.git

fatal: destination path 'GenAI---RAG-using-LangChain' already exists and is not an empty directory.


In [10]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from google.colab import userdata

# Step 1: Set your OpenRouter API key and endpoint
os.environ["OPENAI_API_KEY"] = userdata.get("OPENROUTER_OPENAI") # Replace with your actual OpenRouter API key
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

# Step 2: Initialize the LLM
llm = ChatOpenAI(
    model="mistralai/mistral-7b-instruct",  # or try "meta-llama/llama-3-8b-instruct", etc.
    temperature=0.7,
    openai_api_base=os.environ["OPENAI_API_BASE"],
    openai_api_key=os.environ["OPENAI_API_KEY"],
    request_timeout=60,
    verbose=True
)

# Step 3: Interact with the model (normal chat mode)
while True:
    query = input("\nYou: ")
    if query.lower() in ["exit", "quit", "bye"]:
        print("Goodbye!")
        break

    response = llm([HumanMessage(content=query)])
    print("\nAssistant:", response.content)


/tmp/ipython-input-3908245900.py:11: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(



You: hi


/tmp/ipython-input-3908245900.py:27: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm([HumanMessage(content=query)])



Assistant:  Hello! How can I help you today? If you have any questions or need assistance with something, feel free to ask. I'm here to help!

You: bye
Goodbye!


In [2]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA # RAG - To retrieve the contents from the pdfs or any docs
from langchain.chains.llm import LLMChain  # It's LangChain function to wrap the tools/agents to create a seemless workflow
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate  # To provide the equipped prompt
from langchain_community.document_loaders import PDFPlumberLoader # To read the pdf
from langchain_experimental.text_splitter import SemanticChunker # Text chunks
from langchain_huggingface import HuggingFaceEmbeddings
 # Embed the texts or contents
from langchain_community.vectorstores import FAISS  # Vector DB
from google.colab import userdata
from langchain_experimental.text_splitter import SemanticChunker


In [3]:
# Set your OpenRouter API key and endpoint
os.environ["OPENAI_API_KEY"] = userdata.get("OPENROUTER_OPENAI")
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"  # Endpoint of the openRouter

In [4]:

# Load and process PDF
loader = PDFPlumberLoader("/content/GenAI---RAG-using-LangChain/neural_network.pdf")
docs = loader.load()
print("Pages loaded:", len(docs))

Pages loaded: 224


In [5]:
!pip install sentence-transformers

In [6]:
!pip uninstall -y transformers protobuf
!pip install transformers sentence-transformers

Found existing installation: transformers 4.55.2
Uninstalling transformers-4.55.2:
  Successfully uninstalled transformers-4.55.2
  Using cached transformers-4.55.2-py3-none-any.whl.metadata (41 kB)
Using cached transformers-4.55.2-py3-none-any.whl (11.3 MB)


In [ ]:





# PROCESS 1: CHUNK process
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Semantic splitter with custom
# With SemanticChunker (in LangChain), you don't define chunk_size directly like in other splitters
# (CharacterTextSplitter, RecursiveCharacterTextSplitter, etc.).
# Instead, semantic chunking decides where to split based on semantic similarity between segments, not on fixed sizes.
splitter = SemanticChunker(
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"
    ),
    breakpoint_threshold_type="percentile", # or standard_deviation
    breakpoint_threshold_amount=90, # Value for the threshold (e.g. 95 = 95th percentile distance between embeddings)
    buffer_size=1 # (Optional) Adds context from neighboring chunks
)

# Split document
chunks = splitter.split_documents(docs)
for chunk in chunks:
    print(chunk.page_content)


# PROCESS 2: Vector DB
embedder = HuggingFaceEmbeddings()
vector = FAISS.from_documents(chunks, embedder)
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 2})

'''
# Try with some queries for similarity search
results = retriever.get_relevant_documents("What is the remedy for cold and flu?")

for doc in results:
    print(doc.page_content)
'''

# PROCESS 3: LLM MODEL CHOOSING
# Use ChatOpenAI with OpenRouter backend
llm = ChatOpenAI(
    model="meta-llama/llama-3-8b-instruct",   #"mistralai/mistral-7b-instruct"
    temperature=1.3, # It's a less temp -> so it'll give you some relative information about the content.
    openai_api_base=os.environ["OPENAI_API_BASE"],
    openai_api_key=os.environ["OPENAI_API_KEY"],
    request_timeout=60,
)

# PROCESS 4: PROMPT CREATION
prompt = """
You are a helpful assistant.
Use the following pieces of context to answer the question at the end.
Answer only using the context and be concise (3–4 sentences).

Context: {context}

Question: {question}

Answer:
"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt)


# PROCESS 5: LANGCHAIN CREATION - LLM & PROMPT
llm_chain = LLMChain(llm=llm, prompt=QA_CHAIN_PROMPT, verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)

# DOCUMENT CHAIN PROCESS
combine_documents_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
    document_prompt=document_prompt,
    callbacks=None,
)

# PROCESS 6: WRAPPING ALL TOGETHER ALONG WITH RETRIEVER
qa = RetrievalQA(
    combine_documents_chain=combine_documents_chain,
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
)

# Example query


Neural Networks and Deep Learning
MichaelNielsen
Theoriginalonlinebookcanbefoundat
http://neuralnetworksanddeeplearning.com



(cid:12)
(cid:12) i
(cid:12)
Contents
Whatthisbookisabout iii
Ontheexercisesandproblems v
1 Usingneuralnetstorecognizehandwrittendigits 1
1.1 Perceptrons . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. 2
1.2 Sigmoidneurons . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. 7
1.3 Thearchitectureofneuralnetworks.
.
. . . . . . . . . . . . . . . . . . . . . . .
. 10
1.4 Asimplenetworktoclassifyhandwrittendigits. .
. . . . . . . . . . . . . . . .
. 12
1.5 Learningwithgradientdescent. .
. . . . . . . . . . . . . . . . . . . . . . . . . .
. 15
1.6 Implementingournetworktoclassifydigits . .
. . . . . . . . . . . . . . . . . .
. 24
1.7 Towarddeeplearning .
.
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. 35
2 Howthebackpropagationalgorithmworks 39
2.1 Warmup:afastmatrix-basedapproachtocomputingth

In [ ]:
# Create a chatbot interface
print("Chat with the PDF content (type 'exit', 'quit', or 'bye' to end):")

while True:
    query = input("\nYou: ")
    if query.lower() in ["exit", "quit", "bye"]:
        print("Goodbye!")
        break

    response = qa.invoke({"query": query})
    print("\nAssistant:", response['result'])